In [2]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge requests --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
print("packages installed")

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  26.35 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.8-h422b904_2     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h88dbc4e_2    conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge

libxml2-2.9.8- 100% |################################| Time: 0:00:00  60.78 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  54.05 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  24.34 MB/s
Fetchin

In [3]:
from bs4 import BeautifulSoup
import requests
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors# Matplotlib and associated plotting modules

# import k-means from clustering stage
from sklearn.cluster import KMeans

 # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

 # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [4]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [5]:
import pandas as pd

table= soup.find_all('table')
table = table[0]
trList = table.find_all('tr')
#columsNames = 
i = 0
j = 0
finalList = list()
for tr in trList:
    nestedList = list()
    for td in tr.find_all('td'):
        td_value = str(td.text)
        #print(td.text)
        value = td_value.replace("\n","")
        nestedList.append(value)
    finalList.append(nestedList)


df = pd.DataFrame(finalList[1:],columns=['PostalCode','Borough','Neighborhood'])
   
print (df.head())

  PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront


In [6]:
df_groupby = df.groupby('PostalCode')['Neighborhood'].apply(','.join).reset_index()
#['Borough']['Neighborhood'].apply(','.join)
print(df_groupby.head())

  PostalCode                          Neighborhood
0        M1A                          Not assigned
1        M1B                         Rouge,Malvern
2        M1C  Highland Creek,Rouge Hill,Port Union
3        M1E       Guildwood,Morningside,West Hill
4        M1G                                Woburn


In [7]:
df_merge = pd.merge(df, df_groupby, on='PostalCode')
print(df_merge.head())


  PostalCode           Borough    Neighborhood_x            Neighborhood_y
0        M1A      Not assigned      Not assigned              Not assigned
1        M2A      Not assigned      Not assigned              Not assigned
2        M3A        North York         Parkwoods                 Parkwoods
3        M4A        North York  Victoria Village          Victoria Village
4        M5A  Downtown Toronto      Harbourfront  Harbourfront,Regent Park


In [8]:
df_filter = df_merge[['PostalCode','Borough','Neighborhood_y']]

for index, row in df_filter.iterrows():
    if row['Neighborhood_y'] == 'Not assigned':
        row['Neighborhood_y'] = row['Borough']
print(df_filter.head())

  PostalCode           Borough            Neighborhood_y
0        M1A      Not assigned              Not assigned
1        M2A      Not assigned              Not assigned
2        M3A        North York                 Parkwoods
3        M4A        North York          Victoria Village
4        M5A  Downtown Toronto  Harbourfront,Regent Park


In [9]:
df_cordinates = pd.read_csv('https://cocl.us/Geospatial_data', header = 0)
df_cordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
print(df_cordinates.head())

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476


In [10]:
df_final = pd.merge(df_filter, df_cordinates, on='PostalCode')
df_final.columns = ['PostalCode','Borough','Neighborhood' ,'Latitude', 'Longitude']
print(df_final.head())

  PostalCode           Borough                     Neighborhood   Latitude  \
0        M3A        North York                        Parkwoods  43.753259   
1        M4A        North York                 Victoria Village  43.725882   
2        M5A  Downtown Toronto         Harbourfront,Regent Park  43.654260   
3        M5A  Downtown Toronto         Harbourfront,Regent Park  43.654260   
4        M6A        North York  Lawrence Heights,Lawrence Manor  43.718518   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.360636  
4 -79.464763  


In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [12]:
# create map of Toronto using latitude and longitude values
map_great_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_great_toronto)  
    
map_great_toronto

In [13]:
def getSimilarWord(srtParam):
    if ( 'Toronto' in srtParam ):
        return srtParam
    else:
        return 'None'

df_final['Borough'] = df['Borough'].apply(getSimilarWord)
toronto_data = df_final[df_final['Borough'] != 'None'].reset_index(drop=True)
toronto_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M6A,Downtown Toronto,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
1,M6A,Downtown Toronto,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
2,M9B,Downtown Toronto,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
3,M9B,Downtown Toronto,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
4,M1E,Downtown Toronto,"Guildwood,Morningside,West Hill",43.763573,-79.188711
5,M3H,East Toronto,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259
6,M4H,Downtown Toronto,Thorncliffe Park,43.705369,-79.349372
7,M2J,Downtown Toronto,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
8,M3J,Downtown Toronto,"Northwood Park,York University",43.767980,-79.487262
9,M1K,Downtown Toronto,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
#hide those credentials
CLIENT_ID = 'CWTHDGFKNR3CGR5JFBF1UNFCTUWOP0I4AZDIHDUHF0HX1DMS' # your Foursquare ID
CLIENT_SECRET = 'XPDPKQPVENOV2UFTXVIDYFC42CPYCUYTMO41UXZIGQWER2MF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CWTHDGFKNR3CGR5JFBF1UNFCTUWOP0I4AZDIHDUHF0HX1DMS
CLIENT_SECRET:XPDPKQPVENOV2UFTXVIDYFC42CPYCUYTMO41UXZIGQWER2MF


In [16]:
toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Heights,Lawrence Manor are 43.718517999999996, -79.46476329999999.


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CWTHDGFKNR3CGR5JFBF1UNFCTUWOP0I4AZDIHDUHF0HX1DMS&client_secret=XPDPKQPVENOV2UFTXVIDYFC42CPYCUYTMO41UXZIGQWER2MF&v=20180605&ll=43.718517999999996,-79.46476329999999&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc32871db04f559d9beeaa9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b16e8b6f964a52051bf23e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_boutique_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d104951735',
         'name': 'Boutique',
         'pluralName': 'Boutiques',
         'primary': True,
         'shortName': 'Boutique'}],
       'id': '4b16e8b6f964a52051bf23e3',
       'location': {'address': '120 Orfus Road',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 165,
        'formattedAddress': ['120 Orfus Road', 'Toronto ON M6A 1L9', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7182215,
          'lng': -79.4

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roots,Boutique,43.718221,-79.466776
1,Lac Vien Vietnamese Restaurant,Vietnamese Restaurant,43.721259,-79.468472
2,Kitchen Stuff Plus (Clearance Outlet),Furniture / Home Store,43.719096,-79.462675
3,Orfus Road Shopping Outlets,Clothing Store,43.719045,-79.460849
4,Tim Hortons,Coffee Shop,43.719427,-79.467995


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Lawrence Heights,Lawrence Manor
Lawrence Heights,Lawrence Manor
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Guildwood,Morningside,West Hill
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
CFB Toronto,Downsview East
The Danforth West,Riverdale
Silver Hills,York Mills
Downsview West
The Beaches West,India Bazaar
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Newtonbrook,Willowdale
Downsview Central
Studio District
Bedford Park,Lawrence Manor East
Bedford Park,Lawrence Manor East
Del Ray,Keelesdale,Mount Dennis,Silverthorn
Del Ray,Keelesdale,Mount Dennis,Silverthorn
Roselawn
The Junction North,Runnymede
The Junction North,Runnymede
Weston
Willowdale West
Harbord,University of Toronto
Runnymede,Swansea
Agincourt North,L'Amoreaux East

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Roots,43.718221,-79.466776,Boutique
1,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant
2,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),43.719096,-79.462675,Furniture / Home Store
3,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Orfus Road Shopping Outlets,43.719045,-79.460849,Clothing Store
4,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Tim Hortons,43.719427,-79.467995,Coffee Shop


In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",9,9,9,9,9,9
"Alderwood,Long Branch",18,18,18,18,18,18
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",48,48,48,48,48,48
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CFB Toronto,Downsview East",6,6,6,6,6,6
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",42,42,42,42,42,42
Church and Wellesley,87,87,87,87,87,87


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 193 uniques categories.


In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(890, 193)

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape

(35, 193)

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                       Playground  0.33
1                             Park  0.33
2                 Asian Restaurant  0.33
3  Molecular Gastronomy Restaurant  0.00
4                   Massage Studio  0.00


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2      Pharmacy  0.11
3   Coffee Shop  0.11
4          Pool  0.11


----Bathurst Manor,Downsview North,Wilson Heights----
                  venue  freq
0           Coffee Shop  0.11
1            Restaurant  0.05
2                 Diner  0.05
3  Fast Food Restaurant  0.05
4         Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0           Coffee 

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Asian Restaurant,Creperie,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
1,"Alderwood,Long Branch",Pizza Place,Skating Rink,Coffee Shop,Pharmacy,Gym,Pool,Pub,Sandwich Place,Cosmetics Shop,Dessert Shop
2,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Sushi Restaurant,Pizza Place,Shopping Mall,Diner,Bridal Shop,Restaurant,Bank,Fast Food Restaurant,Supermarket
3,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Women's Store,Dance Studio,Drugstore,Dog Run,Discount Store,Diner
4,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Juice Bar,Japanese Restaurant,Restaurant,Indian Restaurant,Pub,Butcher


In [32]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

In [36]:
# add clustering labels
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6A,Downtown Toronto,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Women's Store,Shoe Store,Coffee Shop,Event Space,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store
1,M6A,Downtown Toronto,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Women's Store,Shoe Store,Coffee Shop,Event Space,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store
2,M9B,Downtown Toronto,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724,2.0,Bank,Women's Store,Event Space,Electronics Store,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
3,M9B,Downtown Toronto,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724,2.0,Bank,Women's Store,Event Space,Electronics Store,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
4,M1E,Downtown Toronto,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Intersection,Medical Center,Mexican Restaurant,Pizza Place,Concert Hall,Comfort Food Restaurant
5,M3H,East Toronto,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259,0.0,Coffee Shop,Sushi Restaurant,Pizza Place,Shopping Mall,Diner,Bridal Shop,Restaurant,Bank,Fast Food Restaurant,Supermarket
6,M4H,Downtown Toronto,Thorncliffe Park,43.705369,-79.349372,0.0,Indian Restaurant,Sandwich Place,Yoga Studio,Supermarket,Grocery Store,Gym,Liquor Store,Coffee Shop,Pharmacy,Pizza Place
7,M2J,Downtown Toronto,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Toy / Game Store,Electronics Store,Asian Restaurant,Tea Room,Restaurant,Japanese Restaurant
8,M3J,Downtown Toronto,"Northwood Park,York University",43.767980,-79.487262,0.0,Coffee Shop,Miscellaneous Shop,Massage Studio,Bar,Furniture / Home Store,Metro Station,Discount Store,Diner,Dessert Shop,Department Store
9,M1K,Downtown Toronto,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Cupcake Shop,Electronics Store,Drugstore,Dog Run,Diner,Dessert Shop


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map Weston Newtonbrook,Willowdale   | toronto_merged['Neighborhood'] != 'Newtonbrook,Willowdale'
markers_colors = [] 

toronto_merged = toronto_merged[toronto_merged['Neighborhood'] != 'Weston']
toronto_merged = toronto_merged[toronto_merged['Neighborhood'] != 'Newtonbrook,Willowdale']
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6A,Downtown Toronto,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Women's Store,Shoe Store,Coffee Shop,Event Space,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store
1,M6A,Downtown Toronto,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Women's Store,Shoe Store,Coffee Shop,Event Space,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store
2,M9B,Downtown Toronto,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724,2.0,Bank,Women's Store,Event Space,Electronics Store,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
3,M9B,Downtown Toronto,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724,2.0,Bank,Women's Store,Event Space,Electronics Store,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
4,M1E,Downtown Toronto,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Intersection,Medical Center,Mexican Restaurant,Pizza Place,Concert Hall,Comfort Food Restaurant
5,M3H,East Toronto,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259,0.0,Coffee Shop,Sushi Restaurant,Pizza Place,Shopping Mall,Diner,Bridal Shop,Restaurant,Bank,Fast Food Restaurant,Supermarket
6,M4H,Downtown Toronto,Thorncliffe Park,43.705369,-79.349372,0.0,Indian Restaurant,Sandwich Place,Yoga Studio,Supermarket,Grocery Store,Gym,Liquor Store,Coffee Shop,Pharmacy,Pizza Place
7,M2J,Downtown Toronto,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Toy / Game Store,Electronics Store,Asian Restaurant,Tea Room,Restaurant,Japanese Restaurant
8,M3J,Downtown Toronto,"Northwood Park,York University",43.767980,-79.487262,0.0,Coffee Shop,Miscellaneous Shop,Massage Studio,Bar,Furniture / Home Store,Metro Station,Discount Store,Diner,Dessert Shop,Department Store
9,M1K,Downtown Toronto,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Cupcake Shop,Electronics Store,Drugstore,Dog Run,Diner,Dessert Shop


In [38]:
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Furniture / Home Store,Clothing Store,Women's Store,Shoe Store,Coffee Shop,Event Space,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store
1,Downtown Toronto,0.0,Furniture / Home Store,Clothing Store,Women's Store,Shoe Store,Coffee Shop,Event Space,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store
4,Downtown Toronto,0.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Intersection,Medical Center,Mexican Restaurant,Pizza Place,Concert Hall,Comfort Food Restaurant
5,East Toronto,0.0,Coffee Shop,Sushi Restaurant,Pizza Place,Shopping Mall,Diner,Bridal Shop,Restaurant,Bank,Fast Food Restaurant,Supermarket
6,Downtown Toronto,0.0,Indian Restaurant,Sandwich Place,Yoga Studio,Supermarket,Grocery Store,Gym,Liquor Store,Coffee Shop,Pharmacy,Pizza Place
7,Downtown Toronto,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Toy / Game Store,Electronics Store,Asian Restaurant,Tea Room,Restaurant,Japanese Restaurant
8,Downtown Toronto,0.0,Coffee Shop,Miscellaneous Shop,Massage Studio,Bar,Furniture / Home Store,Metro Station,Discount Store,Diner,Dessert Shop,Department Store
9,Downtown Toronto,0.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Cupcake Shop,Electronics Store,Drugstore,Dog Run,Diner,Dessert Shop
10,Downtown Toronto,0.0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Women's Store,Dance Studio,Drugstore,Dog Run,Discount Store,Diner
11,Downtown Toronto,0.0,Park,Airport,Bus Stop,Creperie,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,East Toronto,1.0,Baseball Field,Food Truck,Women's Store,Dance Studio,Electronics Store,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
51,Downtown Toronto,1.0,Baseball Field,Construction & Landscaping,Deli / Bodega,Women's Store,Dance Studio,Electronics Store,Drugstore,Dog Run,Discount Store,Diner
52,Downtown Toronto,1.0,Baseball Field,Construction & Landscaping,Deli / Bodega,Women's Store,Dance Studio,Electronics Store,Drugstore,Dog Run,Discount Store,Diner
53,Downtown Toronto,1.0,Baseball Field,Construction & Landscaping,Deli / Bodega,Women's Store,Dance Studio,Electronics Store,Drugstore,Dog Run,Discount Store,Diner


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2.0,Bank,Women's Store,Event Space,Electronics Store,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
3,Downtown Toronto,2.0,Bank,Women's Store,Event Space,Electronics Store,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3.0,Martial Arts Dojo,Cafeteria,Women's Store,Cupcake Shop,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,East Toronto,4.0,Garden,Home Service,Women's Store,Creperie,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
